In [8]:
# coding: utf-8

import time
import shutil
import sys
import io
import os
import csv
import re
import datetime
import mysql.connector
from  mysql.connector import MySQLConnection, Error

# 這麼多defualt是因為如果不重新設定，在使用 reload函數時，print會被印在 terminal上。
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')


"""
把最原始得資料拿來做ETL，頭尾不用的去掉，加入新的欄位名稱。

"""
cs = time.strftime('%Y/%m/%d_%H:%M:%S',time.localtime())
print cs

try:
    # create 'clean' folder  if not exist.
    if not os.path.exists('/Users/Jackie/Desktop/test1107/clean'):
        os.makedirs('/Users/Jackie/Desktop/test1107/clean')
    # from infiark1/ pick only files to do the trick.
    fileslist=[]
    for root,dirs,files in os.walk('/Users/Jackie/Desktop/test1107'):
        for ele in files:
            if '.csv' in ele:
                fileslist.append(root+'/'+ele)
    for f in fileslist:
        with io.open(f, 'r', encoding='utf-8')as rfile:
            ff=[]
            for ele in rfile:
                ff.append(ele)
            # 檔案名字用VD的編號，從檔案內的第一欄抓出來
            fname = ff[2:-2][0].split(',')[0] 
            with io.open('/Users/Jackie/Desktop/test1107/clean/' + fname + '.csv', 'w', encoding='utf-8') as wfile:
                wfile.write(u'''EQIPnumber,location,direct,YMD,hour,laneNumber,\
                addTotal,15Total,truckflow,carflow,scooterflow,avgspeed,avgPercent,avgCarSpace''' + u'\n')
                # 去掉檔案裡上下不用的中文敘述，再把平均速度='-1'的值挑掉不要，最後是把原來檔案內的','跟'\n'去掉。
                for ele in ff[2:-2]:
                    if ele.split(',')[-4] == '-1.0':
                        continue
                    else:
                        sf=[]
                        for el in ele.split(','):
                            if ele.split(',')[3] == el:
                                sf.append(','.join((ele.split(',')[3]).split(' ')))
                            elif (ele.split(',')[0] == 'VSSWC60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6141E0') & (el == "正向放置"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VS7SN60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VSXTK20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VT8SZ60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VTHRR20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VTXQJ20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VU5QC60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6120C0') & (el == "正向放置"):
                                sf.append('東')    
                            elif (ele.split(',')[0] == 'V6130C0') & (el == "正向放置"):
                                sf.append('東') 
                            elif (ele.split(',')[0] == 'V6140C0') & (el == "正向放置"):
                                sf.append('東') 
                            elif (ele.split(',')[0] == 'V6140C1') & (el == "正向放置"):
                                sf.append('東') 
                            elif (ele.split(',')[0] == 'V6360G0') & (el == "正向放置"):
                                sf.append('西') 
                            elif (ele.split(',')[0] == 'V6360G1') & (el == "正向放置"):
                                sf.append('西') 
                            elif (ele.split(',')[0] == 'V6110C0') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'V7230E0') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'V6380E0') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'V7010A0') & (el == "正向放置"):
                                sf.append('北') 
                            elif (ele.split(',')[0] == 'VJERC50') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'VJERC51') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'VJERC52') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'VJVRL10') & (el == "正向放置"):
                                sf.append('北') 
                            elif (ele.split(',')[0] == 'VJVRL11') & (el == "正向放置"):
                                sf.append('北') 
                            elif (ele.split(',')[0] == 'VJVRL40') & (el == "未指定"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VGTPW50') & (el == "正向放置"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VI9QP10') & (el == "正向放置"):
                                sf.append('北')
                            elif (ele.split(',')[0] == 'VI9QP11') & (el == "正向放置"):
                                sf.append('北')
                            elif (ele.split(',')[0] == 'VI9QP12') & (el == "正向放置"):
                                sf.append('北')
                            elif (ele.split(',')[0] == 'VIKQU50') & (el == "正向放置"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VSNVA60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VSQVZ20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VTVXH20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VTYXQ60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VTZY920') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6121A0') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6150C0') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6150C1') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6151C0') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6151C0') & (el == "反向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6160C0') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6160C1') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6330G7') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6330G8') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6340G0') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6340G1') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6350G0') & (el == "正向放置"):
                                sf.append('東')  
                            else:
                                sf.append(el)
                        wfile.write(','.join(sf)[:-2] + '\n')

except:
    with open('/Users/Jackie/Desktop/test1107/ETL.txt','w') as here3:
        here3.write(u'ETL exception'+u'\n')
        here3.write(str(sys.stderr)+u'\n')
        here3.write(str(sys.exc_info()[0])+u'\n')
        here3.write(str(sys.exc_info()[1])+u'\n')
        here3.write(str(sys.exc_info()[2])+u'\n')
        
cs = time.strftime('%Y/%m/%d_%H:%M:%S',time.localtime())
print cs

2016/11/22_19:08:14
2016/11/22_19:17:39


In [7]:
import time
cs = time.strftime('%Y/%m/%d_%H:%M:%S',time.localtime())
print cs

2016/11/22_19:07:12


In [ ]:




# coding: utf-8

import time
import shutil
import sys
import io
import os
import csv
import re
import datetime
import mysql.connector
from  mysql.connector import MySQLConnection, Error

# 這麼多defualt是因為如果不重新設定，在使用 reload函數時，print會被印在 terminal上。
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')


"""
把最原始得資料拿來做ETL，頭尾不用的去掉，加入新的欄位名稱。

"""
try:
    # create 'clean' folder  if not exist.
    if not os.path.exists('/Users/Jackie/Desktop/test1107/clean'):
        os.makedirs('/Users/Jackie/Desktop/test1107/clean')
    # from infiark1/ pick only files to do the trick.
    fileslist=[]
    for root,dirs,files in os.walk('/Users/Jackie/Desktop/test1107'):
        for ele in files:
            if '.csv' in ele:
                fileslist.append(root+'/'+ele)
    for f in fileslist:
        with io.open(f, 'r', encoding='utf-8')as rfile:
            ff=[]
            for ele in rfile:
                ff.append(ele)
            # 檔案名字用VD的編號，從檔案內的第一欄抓出來
            fname = ff[2:-2][0].split(',')[0] 
            with io.open('/Users/Jackie/Desktop/test1107/clean/' + fname + '.csv', 'w', encoding='utf-8') as wfile:
                wfile.write(u'''EQIPnumber,location,direct,YMD,hour,laneNumber,\
                addTotal,15Total,truckflow,carflow,scooterflow,avgspeed,avgPercent,avgCarSpace''' + u'\n')
                # 去掉檔案裡上下不用的中文敘述，再把平均速度='-1'的值挑掉不要，最後是把原來檔案內的','跟'\n'去掉。
                for ele in ff[2:-2]:
                    if ele.split(',')[-4] == '-1.0':
                        continue
                    else:
                        sf=[]
                        for el in ele.split(','):
                            if ele.split(',')[3] == el:
                                sf.append(','.join((ele.split(',')[3]).split(' ')))
                            elif (ele.split(',')[0] == 'VSSWC60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6141E0') & (el == "正向放置"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VS7SN60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VSXTK20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VT8SZ60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VTHRR20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VTXQJ20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VU5QC60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6120C0') & (el == "正向放置"):
                                sf.append('東')    
                            elif (ele.split(',')[0] == 'V6130C0') & (el == "正向放置"):
                                sf.append('東') 
                            elif (ele.split(',')[0] == 'V6140C0') & (el == "正向放置"):
                                sf.append('東') 
                            elif (ele.split(',')[0] == 'V6140C1') & (el == "正向放置"):
                                sf.append('東') 
                            elif (ele.split(',')[0] == 'V6360G0') & (el == "正向放置"):
                                sf.append('西') 
                            elif (ele.split(',')[0] == 'V6360G1') & (el == "正向放置"):
                                sf.append('西') 
                            elif (ele.split(',')[0] == 'V6110C0') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'V7230E0') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'V6380E0') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'V7010A0') & (el == "正向放置"):
                                sf.append('北') 
                            elif (ele.split(',')[0] == 'VJERC50') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'VJERC51') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'VJERC52') & (el == "正向放置"):
                                sf.append('南') 
                            elif (ele.split(',')[0] == 'VJVRL10') & (el == "正向放置"):
                                sf.append('北') 
                            elif (ele.split(',')[0] == 'VJVRL11') & (el == "正向放置"):
                                sf.append('北') 
                            elif (ele.split(',')[0] == 'VJVRL40') & (el == "未指定"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VGTPW50') & (el == "正向放置"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VI9QP10') & (el == "正向放置"):
                                sf.append('北')
                            elif (ele.split(',')[0] == 'VI9QP11') & (el == "正向放置"):
                                sf.append('北')
                            elif (ele.split(',')[0] == 'VI9QP12') & (el == "正向放置"):
                                sf.append('北')
                            elif (ele.split(',')[0] == 'VIKQU50') & (el == "正向放置"):
                                sf.append('南')
                            elif (ele.split(',')[0] == 'VSNVA60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VSQVZ20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VTVXH20') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'VTYXQ60') & (el == "未指定"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'VTZY920') & (el == "未指定"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6121A0') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6150C0') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6150C1') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6151C0') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6151C0') & (el == "反向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6160C0') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6160C1') & (el == "正向放置"):
                                sf.append('東')
                            elif (ele.split(',')[0] == 'V6330G7') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6330G8') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6340G0') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6340G1') & (el == "正向放置"):
                                sf.append('西')
                            elif (ele.split(',')[0] == 'V6350G0') & (el == "正向放置"):
                                sf.append('東')  
                            else:
                                sf.append(el)
                        wfile.write(','.join(sf)[:-2] + '\n')

except:
    with open('/Users/Jackie/Desktop/test1107/ETL.txt','w') as here3:
        here3.write(u'ETL exception'+u'\n')
        here3.write(str(sys.stderr)+u'\n')
        here3.write(str(sys.exc_info()[0])+u'\n')
        here3.write(str(sys.exc_info()[1])+u'\n')
        here3.write(str(sys.exc_info()[2])+u'\n')
